In [ ]:
from app.main.Main import Main

app = Main()
app.code = "VT" # 股票代碼
app.start_date = "2020-01-01" # 起始日期
app.end_date = "2023-01-01" # 結束日期
app.investment_amount = 1000
app.purchase_day = 1
app.is_fund = False
app.using_taiwan_stock_crawl = False
app.using_taiwan_etf_crawl = False
app.using_yf = True
app.consider_tax = True # 考量到美股股息稅

app.launch()

In [ ]:
from app.main.Main import Main

app = Main()
app.code = "VWRA.L" # 股票代碼
app.start_date = "2020-01-01" # 起始日期
app.end_date = "2023-01-01" # 結束日期
app.investment_amount = 1000
app.purchase_day = 1
app.is_fund = False
app.using_taiwan_stock_crawl = False
app.using_taiwan_etf_crawl = False
app.using_yf = True

app.launch()

In [ ]:
from app.main.Main import Main

app = Main()
app.code = "ACDD01" # 安聯台灣大壩基金-A累積型(台幣)
app.start_date = "2013-01-01" # 起始日期
app.end_date = "2023-01-01" # 結束日期
app.investment_amount = 1000
app.purchase_day = 1
app.is_fund = True
app.using_taiwan_stock_crawl = False
app.using_taiwan_etf_crawl = False

app.launch()

In [ ]:
from app.main.Main import Main

app = Main()
app.code = "0050.TW" # 股票代碼
app.start_date = "2013-01-01" # 起始日期
app.end_date = "2023-01-01" # 結束日期
app.investment_amount = 1000
app.purchase_day = 1
app.is_fund = False
app.using_taiwan_stock_crawl = True
app.using_taiwan_etf_crawl = False

app.launch()

In [ ]:
from app.main.Main import Main

app = Main()
app.code = "0056.TW" # 股票代碼
app.start_date = "2013-01-01" # 起始日期
app.end_date = "2023-01-01" # 結束日期
app.investment_amount = 1000
app.purchase_day = 1
app.is_fund = False
app.using_taiwan_stock_crawl = True
app.using_taiwan_etf_crawl = False

app.launch()

In [ ]:
from app.main.Main import Main

app = Main()
app.code = "2884.TW" #玉山金
app.start_date = "2013-01-01" # 起始日期
app.end_date = "2023-01-01" # 結束日期
app.investment_amount = 1000
app.purchase_day = 1
app.is_fund = False
app.using_taiwan_stock_crawl = True
app.using_taiwan_etf_crawl = False

app.launch()

In [ ]:
from app.main.Main import Main

app = Main()
app.code = "5880.TW" # 合庫金
app.start_date = "2013-01-01" # 起始日期
app.end_date = "2023-01-01" # 結束日期
app.investment_amount = 1000
app.purchase_day = 1
app.is_fund = False
app.using_taiwan_stock_crawl = True
app.using_taiwan_etf_crawl = False

app.launch()

In [ ]:
from app.main.Main import Main

app = Main()
app.code = "9941.TW" # 裕融
app.start_date = "2013-01-01" # 起始日期
app.end_date = "2023-01-01" # 結束日期
app.investment_amount = 1000
app.purchase_day = 1
app.is_fund = False
app.using_taiwan_stock_crawl = True
app.using_taiwan_etf_crawl = False

app.launch()

In [ ]:
from app.main.Main import Main

app = Main()
app.code = "2882.TW" # 國泰金
app.start_date = "2013-01-01" # 起始日期
app.end_date = "2023-01-01" # 結束日期
app.investment_amount = 1000
app.purchase_day = 1
app.is_fund = False
app.using_taiwan_stock_crawl = True
app.using_taiwan_etf_crawl = False

app.launch()

In [ ]:
from app.main.Main import Main

app = Main()
app.code = "2330.TW" # 台積電
app.start_date = "2013-01-01" # 起始日期
app.end_date = "2023-01-01" # 結束日期
app.investment_amount = 1000
app.purchase_day = 1
app.is_fund = False
app.using_taiwan_stock_crawl = True
app.using_taiwan_etf_crawl = False

app.launch()

In [ ]:
from app.main.Main import Main

app = Main()
app.code = "2480.TW" # 敦陽科
app.start_date = "2013-01-01" # 起始日期
app.end_date = "2023-01-01" # 結束日期
app.investment_amount = 1000
app.purchase_day = 1
app.is_fund = False
app.using_taiwan_stock_crawl = True
app.using_taiwan_etf_crawl = False

app.launch()

In [ ]:
from app.main.Main import Main

app = Main()
app.code = "2618.TW" # 長榮航空
app.start_date = "2013-01-01" # 起始日期
app.end_date = "2023-01-01" # 結束日期
app.investment_amount = 1000
app.purchase_day = 1
app.is_fund = False
app.using_taiwan_stock_crawl = True
app.using_taiwan_etf_crawl = False

app.launch()

In [ ]:
from app.main.Main import Main

app = Main()
app.start_date = "2022-05-01" # 起始日期
app.end_date = "2023-05-01" # 結束日期
app.investment_amount = 1000
app.purchase_day = 1
app.is_fund = False
app.using_taiwan_stock_crawl = True
app.using_taiwan_etf_crawl = False

code_list =  [
    "0050.TW", # 股票代碼
    "2480.TW", # 敦陽科
    "9941.TW", # 裕融
]
# code_list =  [
#     "0056.TW", # 股票代碼
#     "0050.TW", # 股票代碼
# ]
# code_list =  [
#     "0050.TW", # 股票代碼
#     "2881.TW", # 富邦金
#     "2882.TW", # 國泰金
#     "2884.TW", # 玉山金
#     "2887.TW", # 台新金
#     "5880.TW", # 合庫金
# ]
for code in code_list:
    app.code = code
    app.launch()